### Testing for Interactive use case

In [1]:
import mlflow
from azureml.core import Workspace, Experiment, Environment, Datastore, Dataset, ScriptRunConfig
from azureml.core.runconfig import PyTorchConfiguration
# from azureml.widgets import RunDetails
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.runconfig import PyTorchConfiguration
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies
from IPython.display import clear_output
import time
import platform
# from ray_on_azureml.ray_on_aml import getRay
import sys
sys.path.append("../") # go to parent dir
import importlib


In [17]:
ray.cluster_resources()

{'CPU': 26.0,
 'node:10.0.0.35': 1.0,
 'memory': 77997084266.0,
 'object_store_memory': 35463301936.0,
 'node:10.0.0.18': 1.0,
 'GPU': 1.0,
 'node:10.0.0.11': 1.0,
 'accelerator_type:K80': 1.0,
 'node:10.0.0.33': 1.0,
 'node:10.0.0.12': 1.0,
 'node:10.0.0.34': 1.0}

In [9]:
from src.ray_on_aml.core import Ray_On_AML
ws = Workspace.from_config()
ray_on_aml =Ray_On_AML(ws=ws, compute_cluster ="worker-cpu-v3",additional_pip_packages=['azureml-mlflow', 'torch', 'torchvision','sklearn'])
ray_on_aml.shutdown(end_all_runs=True)
#clear up all active run
_, ray = ray_on_aml.getRay()
ray.cluster_resources()

Get active run  ray_on_aml_1639356958_5ef4cfc6
Run does not exisit, finding active runs to cancel
Cannot shutdown ray
rank returned is  None
rank returned is  None
azureml_py38


2021-12-13 01:00:37,210	ERROR import_thread.py:89 -- ImportThread: Connection closed by server.
2021-12-13 01:00:37,219	ERROR worker.py:1247 -- listen_error_messages_raylet: Connection closed by server.
2021-12-13 01:00:37,220	ERROR worker.py:478 -- print_logs: Connection closed by server.
2021-12-13 01:00:43,075	INFO services.py:1338 -- View the Ray dashboard at http://127.0.0.1:8265


Found existing cluster, use it.


2021-12-13 01:00:59,045	INFO worker.py:842 -- Connecting to existing Ray cluster at address: 10.0.0.11:6379
2021-12-13 01:00:59,172	WARNING worker.py:1219 -- The autoscaler failed with the following error:
Terminated with signal 15
  File "/anaconda/envs/azureml_py38/lib/python3.8/site-packages/ray/autoscaler/_private/monitor.py", line 543, in <module>
    monitor.run()
  File "/anaconda/envs/azureml_py38/lib/python3.8/site-packages/ray/autoscaler/_private/monitor.py", line 439, in run
    self._run()
  File "/anaconda/envs/azureml_py38/lib/python3.8/site-packages/ray/autoscaler/_private/monitor.py", line 346, in _run
    time.sleep(AUTOSCALER_UPDATE_INTERVAL_S)



Waiting: Cluster status is in  Preparing
Waiting: Cluster status is in  Preparing
Waiting: Cluster status is in  Preparing
Waiting: Cluster status is in  Preparing
Waiting: Cluster status is in  Preparing
Waiting: Cluster status is in  Preparing
Waiting: Cluster status is in  Preparing
Waiting: Cluster status is in  Preparing
Waiting: Cluster status is in  Preparing
Waiting: Cluster status is in  Preparing
Waiting: Cluster status is in  Preparing
Waiting: Cluster status is in  Preparing
Waiting: Cluster status is in  Preparing
Waiting: Cluster status is in  Preparing
Waiting: Cluster status is in  Preparing
Waiting: Cluster status is in  Preparing
Waiting: Cluster status is in  Preparing
Waiting: Cluster status is in  Preparing
Waiting: Cluster status is in  Preparing
Waiting: Cluster status is in  Preparing
Waiting: Cluster status is in  Preparing
Waiting: Cluster status is in  Preparing
Waiting: Cluster status is in  Preparing
Waiting: Cluster status is in  Preparing
Waiting: Cluster

{'object_store_memory': 14251860787.0,
 'accelerator_type:K80': 1.0,
 'GPU': 1.0,
 'CPU': 6.0,
 'node:10.0.0.11': 1.0,
 'memory': 28503721575.0}

In [51]:
ray_on_aml.shutdown()
# import ray
# ray.shutdown()
# ray.init()

Get active run  ray_on_aml_1639357246_fafdc42a


### Testing with Dask on Ray

In [10]:
# import ray
# ray.init()
from ray.util.dask import ray_dask_get
import dask
import dask.array as da
import dask.dataframe as dd
import numpy as np
import pandas as pd
dask.config.set(scheduler=ray_dask_get)
d_arr = da.from_array(np.random.randint(0, 1000, size=(256, 256)))

# The Dask scheduler submits the underlying task graph to Ray.
d_arr.mean().compute(scheduler=ray_dask_get)

# Set the scheduler to ray_dask_get in your config so you don't have to
# specify it on each compute call.

df = dd.from_pandas(
    pd.DataFrame(
        np.random.randint(0, 10000, size=(1024, 2)), columns=["age", "grade"]),
    npartitions=2)
df.groupby(["age"]).mean().compute()

# ray.shutdown()

,grade
age,
7,188.0
20,2956.0
22,7300.0
38,9611.0
67,3711.0
...,...
9881,6826.0
9938,9412.0
9954,3235.0


In [4]:
import dask.dataframe as dd

storage_options = {'account_name': 'azureopendatastorage'}
ddf = dd.read_parquet('az://nyctlc/green/puYear=2019/puMonth=*/*.parquet', storage_options=storage_options)
ddf.count().compute()

RayTaskError(NameError): [36mray::dask:('dataframe-count-agg-c779cbec183eaab06c239b02895358aa', 0)[39m (pid=247, ip=10.0.0.33)
  At least one of the input arguments for this task could not be computed:
ray.exceptions.RayTaskError: [36mray::dask:('dataframe-count-chunk-c779cbec183eaab06c239b02895358aa', 0, 6, 0)[39m (pid=191, ip=10.0.0.33)
  At least one of the input arguments for this task could not be computed:
ray.exceptions.RayTaskError: [36mray::dask:('read-parquet-80717453fe38c42d32a261b8d065b376', 6)[39m (pid=192, ip=10.0.0.33)
  File "/anaconda/envs/azureml_py38/lib/python3.8/site-packages/ray/util/dask/scheduler.py", line 350, in dask_task_wrapper
  File "/azureml-envs/azureml_d28debf0c131ba759185828baf634c78/lib/python3.8/site-packages/dask/optimization.py", line 969, in __call__
    return core.get(self.dsk, self.outkey, dict(zip(self.inkeys, args)))
  File "/azureml-envs/azureml_d28debf0c131ba759185828baf634c78/lib/python3.8/site-packages/dask/core.py", line 149, in ge

In [44]:
#dask

# import ray
from ray.util.dask import ray_dask_get
import dask
import dask.array as da
import dask.dataframe as dd
import numpy as np
import pandas as pd
import dask
import dask.dataframe as dd
import matplotlib.pyplot as plt

from datetime import datetime

from azureml.core import Workspace, Dataset, Model
from adlfs import AzureBlobFileSystem
account_key = ws.get_default_keyvault().get_secret("adls7-account-key")
account_name="adlsgen7"
abfs = AzureBlobFileSystem(account_name="adlsgen7",account_key=account_key,  container_name="mltraining")
abfs2 = AzureBlobFileSystem(account_name="azureopendatastorage",  container_name="isdweatherdatacontainer")


storage_options={'account_name': account_name, 'account_key': account_key}

# ddf = dd.read_parquet('az://mltraining/ISDWeatherDelta/year2008', storage_options=storage_options)


data = ray.data.read_parquet(["az://isdweatherdatacontainer/ISDWeather/year=2012/"], filesystem=abfs2)
data1 = ray.data.read_parquet(["az://isdweatherdatacontainer/ISDWeather/year=2010/"], filesystem=abfs2)
data3 = ray.data.read_parquet(["az://isdweatherdatacontainer/ISDWeather/year=2009/"], filesystem=abfs2)
data4 = ray.data.read_parquet(["az://isdweatherdatacontainer/ISDWeather/year=2011/"], filesystem=abfs2)
data5 = ray.data.read_parquet(["az://isdweatherdatacontainer/ISDWeather/year=2013/"], filesystem=abfs2)
data6 = ray.data.read_parquet(["az://isdweatherdatacontainer/ISDWeather/year=2014/"], filesystem=abfs2)

all_data =data.union(data1).union(data2).union(data3).union(data4).union(data5).union(data6)
all_data.count()

Metadata Fetch Progress: 100%|██████████| 16/16 [00:04<00:00,  3.26it/s]


780032264

In [46]:
all_data.groupby("stationName").count()

Sort Sample:  98%|█████████▊| 613/627 [03:04<00:16,  1.19s/it]

(_sample_block pid=7388, ip=10.0.0.12) 


Sort Sample: 100%|██████████| 627/627 [03:32<00:00,  2.95it/s]


TypeError: '<' not supported between instances of 'str' and 'NoneType'

In [50]:
# all_data_dask = all_data.to_dask().describe().compute()

In [49]:
# all_data_dask.compute()

### Testing Ray Tune for distributed ML tunning

In [23]:
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F
# import ray
from ray import tune
from ray.tune.schedulers import ASHAScheduler
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        # In this example, we don't change the model architecture
        # due to simplicity.
        self.conv1 = nn.Conv2d(1, 3, kernel_size=3)
        self.fc = nn.Linear(192, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 3))
        x = x.view(-1, 192)
        x = self.fc(x)
        return F.log_softmax(x, dim=1)
# Change these values if you want the training to run quicker or slower.
EPOCH_SIZE = 512
TEST_SIZE = 256

def train(model, optimizer, train_loader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        # We set this just for the example to run quickly.
        if batch_idx * len(data) > EPOCH_SIZE:
            return
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()


def test(model, data_loader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(data_loader):
            # We set this just for the example to run quickly.
            if batch_idx * len(data) > TEST_SIZE:
                break
            data, target = data.to(device), target.to(device)
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()

    return correct / total
def train_mnist(config):
    # Data Setup
    mnist_transforms = transforms.Compose(
        [transforms.ToTensor(),
         transforms.Normalize((0.1307, ), (0.3081, ))])

    train_loader = DataLoader(
        datasets.MNIST("~/data", train=True, download=True, transform=mnist_transforms),
        batch_size=64,
        shuffle=True)
    test_loader = DataLoader(
        datasets.MNIST("~/data", train=False, transform=mnist_transforms),
        batch_size=64,
        shuffle=True)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = ConvNet()
    model.to(device)

    optimizer = optim.SGD(
        model.parameters(), lr=config["lr"], momentum=config["momentum"])
    for i in range(10):
        train(model, optimizer, train_loader)
        acc = test(model, test_loader)

        # Send the current training result back to Tune
        tune.report(mean_accuracy=acc)

        if i % 5 == 0:
            # This saves the model to the trial directory
            torch.save(model.state_dict(), "./model.pth")
search_space = {
    "lr": tune.sample_from(lambda spec: 10**(-10 * np.random.rand())),
    "momentum": tune.choice([0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09, 0.01])
}

# Uncomment this to enable distributed execution
# ray.shutdown()
# ray.init(address="auto",ignore_reinit_error=True)
# ray.init(address =f'ray://{headnode_private_ip}:10001',allow_multiple=True,ignore_reinit_error=True )
# Download the dataset first
datasets.MNIST("~/data", train=True, download=True)

analysis = tune.run(train_mnist, config=search_space)


2021-12-13 01:24:59,030	WARNING callback.py:114 -- The TensorboardX logger cannot be instantiated because either TensorboardX or one of it's dependencies is not installed. Please make sure you have the latest version of TensorboardX installed: `pip install -U tensorboardx`
2021-12-13 01:24:59,036	WARNING tune.py:570 -- Tune detects GPUs, but no trials are using GPUs. To enable trials to use GPUs, set tune.run(resources_per_trial={'gpu': 1}...) which allows Tune to expose 1 GPU to each trial. You can also override `Trainable.default_resource_request` if using the Trainable API.


Trial name,status,loc,lr,momentum
train_mnist_7ca0b_00000,PENDING,,0.000164303,0.07


(ImplicitFunc pid=88813) /anaconda/envs/azureml_py38/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448234945/work/c10/core/TensorImpl.h:1156.)
(ImplicitFunc pid=88813)   return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Result for train_mnist_7ca0b_00000:
  date: 2021-12-13_01-25-01
  done: false
  experiment_id: c7e07e291ce54faab67cbcae24560f7e
  hostname: nc6v0
  iterations_since_restore: 1
  mean_accuracy: 0.115625
  node_ip: 10.0.0.11
  pid: 88813
  time_since_restore: 0.2597367763519287
  time_this_iter_s: 0.2597367763519287
  time_total_s: 0.2597367763519287
  timestamp: 1639358701
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 7ca0b_00000
  
Result for train_mnist_7ca0b_00000:
  date: 2021-12-13_01-25-03
  done: true
  experiment_id: c7e07e291ce54faab67cbcae24560f7e
  experiment_tag: 0_lr=0.0001643,momentum=0.07
  hostname: nc6v0
  iterations_since_restore: 10
  mean_accuracy: 0.125
  node_ip: 10.0.0.11
  pid: 88813
  time_since_restore: 2.1971499919891357
  time_this_iter_s: 0.21480560302734375
  time_total_s: 2.1971499919891357
  timestamp: 1639358703
  timesteps_since_restore: 0
  training_iteration: 10
  trial_id: 7ca0b_00000
  


Trial name,status,loc,lr,momentum,acc,iter,total time (s)
train_mnist_7ca0b_00000,TERMINATED,10.0.0.11:88813,0.000164303,0.07,0.125,10,2.19715


2021-12-13 01:25:03,182	INFO tune.py:626 -- Total run time: 4.22 seconds (4.03 seconds for the tuning loop).


In [26]:
 import sklearn.datasets
 import sklearn.metrics
 from sklearn.model_selection import train_test_split
 import xgboost as xgb

 from ray import tune


 def train_breast_cancer(config):
     # Load dataset
     data, labels = sklearn.datasets.load_breast_cancer(return_X_y=True)
     # Split into train and test set
     train_x, test_x, train_y, test_y = train_test_split(
         data, labels, test_size=0.25)
     # Build input matrices for XGBoost
     train_set = xgb.DMatrix(train_x, label=train_y)
     test_set = xgb.DMatrix(test_x, label=test_y)
     # Train the classifier
     results = {}
     xgb.train(
         config,
         train_set,
         evals=[(test_set, "eval")],
         evals_result=results,
         verbose_eval=False)
     # Return prediction accuracy
     accuracy = 1. - results["eval"]["error"][-1]
     tune.report(mean_accuracy=accuracy, done=True)


 config = {
     "objective": "binary:logistic",
     "eval_metric": ["logloss", "error"],
     "max_depth": tune.randint(1, 9),
     "min_child_weight": tune.choice([1, 2, 3, 4, 5,6,7]),
     "subsample": tune.uniform(0.1, 1.0),
     "eta": tune.loguniform(1e-6, 1e-1)
 }
 analysis = tune.run(
     train_breast_cancer,
     resources_per_trial={"cpu": 0.5},
     config=config,
     num_samples=1200)


2021-12-13 01:26:24,973	WARNING callback.py:114 -- The TensorboardX logger cannot be instantiated because either TensorboardX or one of it's dependencies is not installed. Please make sure you have the latest version of TensorboardX installed: `pip install -U tensorboardx`
2021-12-13 01:26:24,978	WARNING tune.py:570 -- Tune detects GPUs, but no trials are using GPUs. To enable trials to use GPUs, set tune.run(resources_per_trial={'gpu': 1}...) which allows Tune to expose 1 GPU to each trial. You can also override `Trainable.default_resource_request` if using the Trainable API.


Trial name,status,loc,eta,max_depth,min_child_weight,subsample
train_breast_cancer_afdd3_00000,PENDING,,7.21882e-06,6,7,0.991635
train_breast_cancer_afdd3_00001,PENDING,,0.0896394,8,6,0.454755
train_breast_cancer_afdd3_00002,PENDING,,2.57277e-05,3,4,0.140229
train_breast_cancer_afdd3_00003,PENDING,,1.97735e-06,5,5,0.403903
train_breast_cancer_afdd3_00004,PENDING,,0.020118,2,1,0.59187
train_breast_cancer_afdd3_00005,PENDING,,3.74366e-05,1,7,0.24934
train_breast_cancer_afdd3_00006,PENDING,,2.09181e-05,1,7,0.488391
train_breast_cancer_afdd3_00007,PENDING,,1.71127e-06,4,6,0.214962
train_breast_cancer_afdd3_00008,PENDING,,0.00117104,4,3,0.905878
train_breast_cancer_afdd3_00009,PENDING,,2.69995e-05,6,1,0.751093


Result for train_breast_cancer_afdd3_00005:
  date: 2021-12-13_01-26-28
  done: true
  experiment_id: 71548abc7350478c8a8e6e2b125d53b6
  hostname: d79d83b7a9c34428858da4e239f4a6e8000006
  iterations_since_restore: 1
  mean_accuracy: 0.923077
  node_ip: 10.0.0.34
  pid: 1328
  time_since_restore: 1.2593162059783936
  time_this_iter_s: 1.2593162059783936
  time_total_s: 1.2593162059783936
  timestamp: 1639358788
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: afdd3_00005
  
Result for train_breast_cancer_afdd3_00004:
  date: 2021-12-13_01-26-28
  done: true
  experiment_id: c1316ead82f24f2a9216a041f06727e4
  hostname: nc6v0
  iterations_since_restore: 1
  mean_accuracy: 0.916084
  node_ip: 10.0.0.11
  pid: 90218
  time_since_restore: 0.10287618637084961
  time_this_iter_s: 0.10287618637084961
  time_total_s: 0.10287618637084961
  timestamp: 1639358788
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: afdd3_00004
  
Result for train_breast_cancer_afdd3_0001

Trial name,status,loc,eta,max_depth,min_child_weight,subsample,acc,iter,total time (s)
train_breast_cancer_afdd3_00000,RUNNING,10.0.0.11:90217,7.21882e-06,6,7,0.991635,,,
train_breast_cancer_afdd3_00002,RUNNING,10.0.0.11:90216,2.57277e-05,3,4,0.140229,,,
train_breast_cancer_afdd3_00006,RUNNING,10.0.0.11:90215,2.09181e-05,1,7,0.488391,,,
train_breast_cancer_afdd3_00007,RUNNING,10.0.0.11:90383,1.71127e-06,4,6,0.214962,,,
train_breast_cancer_afdd3_00009,RUNNING,10.0.0.33:1380,2.69995e-05,6,1,0.751093,,,
train_breast_cancer_afdd3_00015,RUNNING,10.0.0.35:1393,0.000281124,7,4,0.987052,,,
train_breast_cancer_afdd3_00017,RUNNING,10.0.0.34:1455,0.000134132,3,7,0.676895,,,
train_breast_cancer_afdd3_00065,PENDING,,4.15447e-06,5,3,0.278803,,,
train_breast_cancer_afdd3_00066,PENDING,,0.0128476,4,3,0.309692,,,
train_breast_cancer_afdd3_00067,PENDING,,1.55758e-06,8,6,0.408838,,,


Result for train_breast_cancer_afdd3_00038:
  date: 2021-12-13_01-26-35
  done: true
  experiment_id: b52d334425714097a4d25434c899dcda
  hostname: nc6v0
  iterations_since_restore: 1
  mean_accuracy: 0.93007
  node_ip: 10.0.0.11
  pid: 90225
  time_since_restore: 4.650930166244507
  time_this_iter_s: 4.650930166244507
  time_total_s: 4.650930166244507
  timestamp: 1639358795
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: afdd3_00038
  


...

Result for train_breast_cancer_afdd3_01148:
  date: 2021-12-13_01-29-56
  done: true
  experiment_id: 78b36c3c543541ccb45dc1e7ee209843
  hostname: d79d83b7a9c34428858da4e239f4a6e800000A
  iterations_since_restore: 1
  mean_accuracy: 0.944056
  node_ip: 10.0.0.18
  pid: 6995
  time_since_restore: 0.679140567779541
  time_this_iter_s: 0.679140567779541
  time_total_s: 0.679140567779541
  timestamp: 1639358996
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: afdd3_01148
  
Result for train_breast_cancer_afdd3_01141:
  date: 2021-12-13_01-29-55
  done: true
  experiment_id: 99f4ceb7609d4383b41ab3e253b1bc45
  hostname: d79d83b7a9c34428858da4e239f4a6e800000A
  iterations_since_restore: 1
  mean_accuracy: 0.923077
  node_ip: 10.0.0.18
  pid: 6993
  time_since_restore: 0.6956658363342285
  time_this_iter_s: 0.6956658363342285
  time_total_s: 0.6956658363342285
  timestamp: 1639358995
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: afdd3_01141
  
Result for trai

Trial name,status,loc,eta,max_depth,min_child_weight,subsample,acc,iter,total time (s)
train_breast_cancer_afdd3_01158,RUNNING,10.0.0.12:7135,6.54236e-05,4,7,0.34472,,,
train_breast_cancer_afdd3_01159,RUNNING,10.0.0.18:7117,0.000528933,6,3,0.582434,,,
train_breast_cancer_afdd3_01160,RUNNING,10.0.0.18:7118,0.000333722,4,7,0.248455,,,
train_breast_cancer_afdd3_01161,RUNNING,10.0.0.18:7121,0.000249534,7,1,0.831787,,,
train_breast_cancer_afdd3_01162,RUNNING,10.0.0.33:7033,2.02373e-05,7,2,0.729797,,,
train_breast_cancer_afdd3_01163,RUNNING,10.0.0.33:7035,2.33442e-06,7,5,0.952891,,,
train_breast_cancer_afdd3_01164,RUNNING,10.0.0.35:7319,0.000176917,8,1,0.460494,,,
train_breast_cancer_afdd3_01165,RUNNING,10.0.0.11:98392,0.000187452,3,5,0.550263,,,
train_breast_cancer_afdd3_01166,RUNNING,10.0.0.35:7321,3.63468e-06,5,5,0.244264,,,
train_breast_cancer_afdd3_01167,RUNNING,10.0.0.35:7323,6.994e-05,2,4,0.422623,,,


Result for train_breast_cancer_afdd3_01186:
  date: 2021-12-13_01-30-02
  done: true
  experiment_id: 4fe2df6ef4624f6cad59b3119ed88d3a
  hostname: d79d83b7a9c34428858da4e239f4a6e8000006
  iterations_since_restore: 1
  mean_accuracy: 0.916084
  node_ip: 10.0.0.34
  pid: 7176
  time_since_restore: 1.3054296970367432
  time_this_iter_s: 1.3054296970367432
  time_total_s: 1.3054296970367432
  timestamp: 1639359002
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: afdd3_01186
  
Result for train_breast_cancer_afdd3_01179:
  date: 2021-12-13_01-30-03
  done: true
  experiment_id: 65e5aa044fd442568e948cd884171448
  hostname: d79d83b7a9c34428858da4e239f4a6e800000A
  iterations_since_restore: 1
  mean_accuracy: 0.937063
  node_ip: 10.0.0.18
  pid: 7200
  time_since_restore: 1.3143863677978516
  time_this_iter_s: 1.3143863677978516
  time_total_s: 1.3143863677978516
  timestamp: 1639359003
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: afdd3_01179
  
Result for t

Trial name,status,loc,eta,max_depth,min_child_weight,subsample,acc,iter,total time (s)
train_breast_cancer_afdd3_00000,TERMINATED,10.0.0.11:90217,7.21882e-06,6,7,0.991635,0.867133,1,3.28239
train_breast_cancer_afdd3_00001,TERMINATED,10.0.0.34:1329,0.0896394,8,6,0.454755,0.958042,1,2.20876
train_breast_cancer_afdd3_00002,TERMINATED,10.0.0.11:90216,2.57277e-05,3,4,0.140229,0.937063,1,2.31878
train_breast_cancer_afdd3_00003,TERMINATED,10.0.0.35:1390,1.97735e-06,5,5,0.403903,0.909091,1,2.59442
train_breast_cancer_afdd3_00004,TERMINATED,10.0.0.11:90218,0.020118,2,1,0.59187,0.916084,1,0.102876
train_breast_cancer_afdd3_00005,TERMINATED,10.0.0.34:1328,3.74366e-05,1,7,0.24934,0.923077,1,1.25932
train_breast_cancer_afdd3_00006,TERMINATED,10.0.0.11:90215,2.09181e-05,1,7,0.488391,0.895105,1,1.49317
train_breast_cancer_afdd3_00007,TERMINATED,10.0.0.11:90383,1.71127e-06,4,6,0.214962,0.937063,1,2.97856
train_breast_cancer_afdd3_00008,TERMINATED,10.0.0.18:1369,0.00117104,4,3,0.905878,0.937063,1,2.24375
train_breast_cancer_afdd3_00009,TERMINATED,10.0.0.33:1380,2.69995e-05,6,1,0.751093,0.93007,1,3.10566


2021-12-13 01:30:05,811	INFO tune.py:626 -- Total run time: 220.88 seconds (220.12 seconds for the tuning loop).


### Testing Spark on Ray

In [1]:
import ray
import raydp
import os
ray.shutdown()
ray.init()
os.environ["PYSPARK_PYTHON"]="/anaconda/envs/azureml_py38/bin/python3"

# ray.init(address ='ray://10.0.0.11:6379')
spark = raydp.init_spark(
  app_name = "example",
  num_executors = 2,
  executor_cores = 1,
  executor_memory = "1gb"
)

# data =spark.read.format("csv").option("header", True).load("wasbs://ojsales-simulatedcontainer@azureopendatastorage.blob.core.windows.net/oj_sales_data/Store10*.csv")


# # normal data processesing with Spark
# df = spark.createDataFrame([('look',), ('spark',), ('tutorial',), ('spark',), ('look', ), ('python', )], ['word'])
# df.show()
# word_count = df.groupBy('word').count()
# word_count.show()
import pandas as pd

from pyspark.sql.functions import col, pandas_udf
from pyspark.sql.types import LongType

# Declare the function and create the UDF
def multiply_func(a: pd.Series, b: pd.Series) -> pd.Series:
    return a * b

multiply = pandas_udf(multiply_func, returnType=LongType())

# The function for a pandas_udf should be able to execute with local Pandas data
x = pd.Series([1, 2, 3])
print(multiply_func(x, x))
# 0    1
# 1    4
# 2    9
# dtype: int64

# Create a Spark DataFrame, 'spark' is an existing SparkSession
df = spark.createDataFrame(pd.DataFrame(x, columns=["x"]))

# Execute function as a Spark vectorized UDF
df.select(multiply(col("x"), col("x"))).show()
# +-------------------+
# |multiply_func(x, x)|
# +-------------------+
# |                  1|
# |                  4|
# |                  9|
# +-------------------+


# stop the spark cluster
raydp.stop_spark()


2021-12-06 19:48:42,187	INFO services.py:1338 -- View the Ray dashboard at http://127.0.0.1:8266


2021-12-06 19:48:45,043 WARN Utils [Thread-1]: Your hostname, nc6v0 resolves to a loopback address: 127.0.0.1; using 10.0.0.11 instead (on interface eth0)
2021-12-06 19:48:45,046 WARN Utils [Thread-1]: Set SPARK_LOCAL_IP if you need to bind to another address
2021-12-06 19:48:45,240 WARN NativeCodeLoader [Thread-1]: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


2021-12-06 19:48:45,321 INFO SecurityManager [Thread-1]: Changing view acls to: azureuser
2021-12-06 19:48:45,321 INFO SecurityManager [Thread-1]: Changing modify acls to: azureuser
2021-12-06 19:48:45,322 INFO SecurityManager [Thread-1]: Changing view acls groups to: 
2021-12-06 19:48:45,322 INFO SecurityManager [Thread-1]: Changing modify acls groups to: 
2021-12-06 19:48:45,323 INFO SecurityManager [Thread-1]: SecurityManager: authentication disabled; ui acls disabled; users  with view permissions: Set(azureuser); groups with view permissions: Set(); users  with modify permissions: Set(azureuser); groups with modify permissions: Set()
2021-12-06 19:48:45,591 INFO Utils [Thread-1]: Successfully started service 'RAY_RPC_ENV' on port 44621.


SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/anaconda/envs/azureml_py38/lib/python3.8/site-packages/ray/jars/ray_dist.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/anaconda/envs/azureml_py38/lib/python3.8/site-packages/pyspark/jars/slf4j-log4j12-1.7.30.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]


2021-12-06 19:48:48,798 INFO RayAppMaster$RayAppMasterEndpoint [dispatcher-event-loop-1]: Registering app example
2021-12-06 19:48:48,802 INFO RayAppMaster$RayAppMasterEndpoint [dispatcher-event-loop-1]: Registered app example with ID app-20211206194848-0000
0    1
1    4
2    9
dtype: int64


(raylet) WARNING: An illegal reflective access operation has occurred
(raylet) WARNING: Illegal reflective access by org.apache.spark.unsafe.Platform (file:/anaconda/envs/azureml_py38/lib/python3.8/site-packages/pyspark/jars/spark-unsafe_2.12-3.2.0.jar) to constructor java.nio.DirectByteBuffer(long,int)
(raylet) WARNING: Please consider reporting this to the maintainers of org.apache.spark.unsafe.Platform
(raylet) WARNING: Use --illegal-access=warn to enable warnings of further illegal reflective access operations
(raylet) WARNING: All illegal access operations will be denied in a future release
(raylet) WARNING: An illegal reflective access operation has occurred
(raylet) WARNING: Illegal reflective access by org.apache.spark.unsafe.Platform (file:/anaconda/envs/azureml_py38/lib/python3.8/site-packages/pyspark/jars/spark-unsafe_2.12-3.2.0.jar) to constructor java.nio.DirectByteBuffer(long,int)
(raylet) WARNING: Please consider reporting this to the maintainers of org.apache.spark.unsa

+-------------------+
|multiply_func(x, x)|
+-------------------+
|                  1|
|                  4|
|                  9|
+-------------------+

2021-12-06 19:48:59,086 INFO RayAppMaster [Thread-1]: Stopping RayAppMaster


In [ ]:
raydp.stop_spark()


## Testing Ray on Job Cluster

In [15]:
#     pyarrow >=6.0.1
# dask >=2021.11.2
# adlfs >=2021.10.0
# fsspec==2021.10.1
# ray[default]==1.9.0
ws = Workspace.from_config()

# base_conda_dep =['adlfs>=2021.10.0','pytorch','matplotlib','torchvision','pip']
# base_pip_dep = ['sklearn','xgboost','lightgbm','ray[default]==1.9.0', 'xgboost_ray', 'dask','pyarrow>=6.0.1', 'azureml-mlflow']
compute_cluster = 'worker-cpu-v3'
maxnode =5
vm_size='STANDARD_DS3_V2'
vnet='rayvnet'
subnet='default'
exp ='ray_on_aml_job'
ws_detail = ws.get_details()
ws_rg = ws_detail['id'].split("/")[4]
vnet_rg=None
try:
    ray_cluster = ComputeTarget(workspace=ws, name=compute_cluster)

    print('Found existing cluster, use it.')
except ComputeTargetException:
    if vnet_rg is None:
        vnet_rg = ws_rg
    compute_config = AmlCompute.provisioning_configuration(vm_size=vm_size,
                                                        min_nodes=0, max_nodes=maxnode,
                                                        vnet_resourcegroup_name=vnet_rg,
                                                        vnet_name=vnet,
                                                        subnet_name=subnet)
    ray_cluster = ComputeTarget.create(ws, compute_cluster, compute_config)

    ray_cluster.wait_for_completion(show_output=True)


# python_version = ["python="+platform.python_version()]



# conda_packages = python_version+base_conda_dep
# pip_packages = base_pip_dep 

# conda_dep = CondaDependencies()

# rayEnv = Environment(name="rayEnv")
rayEnv = Environment.get(ws, "rayEnv", version=16)
# for conda_package in conda_packages:
#     conda_dep.add_conda_package(conda_package)

# for pip_package in pip_packages:
#     conda_dep.add_pip_package(pip_package)

# # Adds dependencies to PythonSection of myenv
# rayEnv.python.conda_dependencies=conda_dep

src = ScriptRunConfig(source_directory='job',
                script='aml_job.py',
                environment=rayEnv,
                compute_target=ray_cluster,
                distributed_job_config=PyTorchConfiguration(node_count=maxnode),
                    # arguments = ["--master_ip",master_ip]
                )
run = Experiment(ws, exp).submit(src)

Found existing cluster, use it.
